In [ ]:
import requests
from urllib.parse import urlparse, parse_qs
import json
import polyline
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import geocoder
import numpy as np
import folium
import base64
import os
import time
from faker import Factory

In [ ]:
# First, successfully requesting any athletes' past activities (even your own) requires an token with the enhanced scope 'activity:read_all'. 
# To generate a token with this scope, the user must authenticate with strava through your app via a post request with a redirect. 
# If you don't want to set up a web server, the athelete can put the below url in their browser. 
### Be sure to replace ['client_id'] with their app's client ID found at https://www.strava.com/settings/api

# https://www.strava.com/oauth/authorize?client_id=[client_id]&redirect_uri=http://localhost&response_type=code&grant_type=authorization_code&scope=activity:read_all

# You will be directed to a strava athentication page asking to give the app permission to your private activities
# Click 'Authorize'
# You will be redirected to a page that doesn't exist but the url will contain the eenhanced scope authorization code needed
# Set the response variable below to the redirect url, it should look like this 'http://localhost/?state=&code=[your_code_here]&scope=read,activity:read_all'
    
redirect = ''

# Next we will pasre the redirect url and set the authorization code to a variable that can be used in the request

# Using the modules urlparse and parse_qs from the urllib library, the below code parses the redirect url, queries for the authorization code and sets it to the variable `authorization_code`

parsed_url = urlparse(redirect)
authorization_code = parse_qs(parsed_url.query)['code'][0]
print(authorization_code)

# Now you can make a request to the Strava API for the Athlete's token with the scope necessary
# But first, we must set the necessary variables to make the call
# Again, these can mostly be found here: https://www.strava.com/settings/api

token_url ='https://www.strava.com/api/v3/oauth/token'
client_id =  # This is the client id for your app
client_secret =  # This is the secret code for your app, keep it safe
grant_type = 'authorization_code'

# Once the variables are set above, you can make the post request and set it to the variable token_response

token_response = requests.post(token_url, data={'client_id':client_id,'client_secret':client_secret,'code':authorization_code,'grant_type':grant_type})


In [ ]:
# Now that we have the reponse saved as a variable, we can convert to json and retrive our token
json_token_response = json.loads(token_response.text)

# We can easily do this by calling the keys of the values we want from the json response and setting them as variables
access_token = json_token_response['access_token']
athelete_id = json_token_response['athlete']['id']

print(athelete_id)
print(access_token)


In [ ]:
# Now we can request the athelete's saved routes
# But what if they have A LOT of saved routes? The api defaults to 200 per page?
# We can create a function to make the request and call it in a for loop to iterate through multiple pages of routes

# get_routes function that takes access token, results per page, and page number
# 200 is the default as it is the maximum number of routes that can be returned for a page

def get_routes(access_token, athelete_id, per_page=200, page=1):
    routes_url = 'https://www.strava.com/api/v3/athletes/' + str(athelete_id) + '/routes'
    data = {'access_token':access_token, 'per_page':per_page, 'page':page}
    
    response = requests.get(
        routes_url,
        data=data
    )
    
    return response

# Now we can iterate through our routes catalog
# Since each request returns a list of json results, we'll nest another for loop to pull each route and load them to our list individually

max_pages = 3
data = []
for page_number in range(1, max_pages + 1):
    page_data = json.loads(get_routes(access_token, athelete_id, page=page_number).text)
    for route in page_data:
        data.append(route)
    if page_data == []:
        break

#route_url = 'https://www.strava.com/api/v3/athletes/' + str(athelete_id) + '/routes'
#routes_response = requests.get(route_url, data={'access_token':access_token, 'per_page':200})

In [ ]:
# Then we will create our own route list of dictionaries, filtered with only the details we need

# Start by initializing route_list
route_list = []

# Then iterate over our json_routes_response variable using a for loop to build dictionaries that are then appended to our route_list
for route in data:
    route_details = {}
    route_details['id'] = route['id']
    route_details['name'] = route['name']
    route_details['description'] = route['description']
    route_details['timestamp'] = route['timestamp']
    route_details['estimated_moving_time'] = route['estimated_moving_time']
    route_details['distance'] = round((route['distance'] / 1000),2)
    route_details['elevation_gain'] = round(route['elevation_gain'], 2)
    route_details['summary_polyline'] = route['map']['summary_polyline']
    route_list.append(route_details.copy())

    
    
    

In [ ]:
# Now save it as a pandas DataFrame
routes_df = pd.DataFrame(route_list)

In [ ]:
# Next we'll use the .decode() method to decode the summary_polyline into a list of lat/long tuples (i.e. polyline data) and save it for each route in the DataFrame
routes_df['polyline'] = routes_df['summary_polyline'].apply(polyline.decode)

In [ ]:
# For the sake of efficiency, we'll want to only plot a subset of the routes
# The best way will be to filter the list by Geo
# To start, we can take the first lat,long tuple in the polyline to get our starting coordinates
# Then we can reverse geocode to get a list of the states in which the route starts and append it to the DataFrame

# Using list comprehension, we'll make a list of starting coordinates for each route
start_latlong = [route[0] for route in routes_df['polyline']]

# We initialize start_state list
start_state = []

# We loop through the starting coordinates and translate them to a state using reverse geocoding
for coord in start_latlong:
    g = geocoder.arcgis([coord[0], coord[1]], method='reverse').state
    start_state.append(g)
    
# Now we can add our list of states to our DataFrame as a column
routes_df['state'] = start_state


In [ ]:
# We'll create a mask to filter for only routes starting in California
ca_mask = routes_df['state'] == 'California'

# We'll call the California mask against the original DataFrame and save a copy to a new DataFrame
ca_routes_df = routes_df[ca_mask].copy()

In [ ]:
# Now we are ready to plot our first route!
# Initialize my_route with the first route in my california routes DataFrame
my_route = ca_routes_df.iloc[0,:]

# define the centroid (i.e. center position of the route where the map should focus) 
centroid = [
    np.mean([coord[0] for coord in my_route['polyline']]),
    np.mean([coord[1] for coord in my_route['polyline']])
]

# Use folium library to plot my_route on the map 
m = folium.Map(location=centroid, zoom_start=12)
folium.PolyLine(my_route['polyline'], color='red').add_to(m)
display(m)

In [ ]:
# For better route evaluation, it's always helpful to get a sense of elevation
# Let's produce an elevation profile for each route
# We can get route elevation by translating the polyline coordinates using the open-elevation api
# The below function will help do this efficiently 

def get_elevation (lat, long):
    url_endpoint = 'https://api.open-elevation.com/api/v1/lookup'
    parameters = {'locations': f'{lat}, {long}'}
    response = requests.get(url_endpoint,params=parameters).json()['results'][0]
    return response['elevation']    



In [ ]:
# Now, using the index of the DataFrame, we can iterate through each record and generate elevation values for each route's polyline
# We'll save our responses as a list of lists that we can append to our DataFrame 

## Warning, this operation can easily timeout depending on the size of your list and/or routes - good thing we filtered our down!

elevations = []
for idx in tqdm(ca_routes_df.index):
    route = ca_routes_df.loc[idx, :]
    # List Comprehension calling our function
    elevation = [get_elevation(coord[0],coord[1]) for coord in route['polyline']]
    elevations.append(elevation)
    
ca_routes_df['elevation'] = elevations

In [ ]:
# With every route now having elevation, let's plot each one using matplotlib
# We'll save them as .pngs so we can call and view them once we've plotted all of our routes

# Initialize our elevation_profile dictionary
elevation_profile = {}

# Loop through our DataFrame
for row in ca_routes_df.iterrows():
    # Ignore the index, set the data Series to row_values 
    row_values = row[1]
    
    # Create a figure, plotting the moving average of the elevation 
    fig, ax = plt.subplots(figsize=(6,2))
    ax = pd.Series(row_values['elevation']).rolling(3).mean().plot(
        ax=ax,
        color='red',
        legend=False
    )
    ax.set_ylabel('Elevation')
    ax.axes.xaxis.set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    png = 'elevation_profile_{}.png'.format(row_values['id'])
    fig.savefig(png, dpi=75)
    plt.close()
    
    # read png file
    elevation_profile[row_values['id']] = base64.b64encode(open(png, 'rb').read()).decode()
    
    #delete file
    os.remove(png)
    

In [ ]:
# plot all routes on map
resolution, width, height = 75, 6, 6.5

# function to create centroids 
def centroid(polylines):
    x, y = [], []
    for polyline in polylines:
        for coord in polyline:
            x.append(coord[0])
            y.append(coord[1])
    return [(min(x)+max(x))/2, (min(y)+max(y))/2]


#new folium map
m = folium.Map(location=centroid(ca_routes_df['polyline']), zoom_start=8)

#iterate through routes
for row in ca_routes_df.iterrows():
    fake = Factory.create()
    color = fake.hex_color()
    row_index = row[0]
    row_values = row[1]
    folium.PolyLine(row_values['polyline'], color=color).add_to(m)

    #halfway coords for popup
    halfway_coord = row_values['polyline'][int(len(row_values['polyline'])/2)]


    
    # popup text
    html = """
    <h3>{}</h3>
        <p>
            <code>
            Description : {} 
            </code>
        </p>
    <h4>Details</h4>
        <p> 
            <code>
                Distance&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp: {:.2f} km <br>
                Elevation Gain&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp: {} m <br>
                Est. Moving Time&nbsp;&nbsp;&nbsp;&nbsp: {} <br>
                https://www.strava.com/routes/{} <br>
            </code>
        </p>
    <img src="data:image/png;base64,{}">
    """.format(
        row_values['name'], 
        row_values['description'],  
        row_values['distance'], 
        row_values['elevation_gain'], 
        time.strftime('%H:%M:%S', time.gmtime(row_values['estimated_moving_time'])),  
        row_values['id'], 
        elevation_profile[row_values['id']], 
    )
    
    # add marker to map
    iframe = folium.IFrame(html, width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='white', icon_color = color, icon='map-pin', prefix='fa')
    marker = folium.Marker(location=halfway_coord, popup=popup, icon=icon)
    marker.add_to(m)

m.save('mymap.html')
display(m)